<a href="https://colab.research.google.com/github/VincentZuo/NasdaqJuggernaut/blob/master/Tf_CNN_LSTM_BiDiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q tensorflow tensorflow-datasets

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Read data using pandas

In [0]:
data_url = "https://raw.githubusercontent.com/VincentZuo/NasdaqJuggernaut/master/upliftingNews_td%20-%20upliftingNews_td.csv.csv"

TARGET_COLS = ["close2close", "open2open", "overnight"]
TARGET_COL = TARGET_COLS[0]
SENTENCE_COL = "text"

df = pd.read_csv(data_url)

Encode data using news encoding

In [6]:
_, cnn_dailymail_info = tfds.load('cnn_dailymail/subwords32k', with_info=True,
                          as_supervised=True)

encoder = cnn_dailymail_info.features['article'].encoder

df["embedding"] = df["text"].map(lambda text : np.asarray(encoder.encode(text)))

df.head()

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset cnn_dailymail downloaded and prepared to /root/tensorflow_datasets/cnn_dailymail/subwords32k/0.0.2. Subsequent calls will reuse this data.


,text,overnight,open2open,close2close,embedding
0,Mother donates 92 gallons of breast milk follo...,1,1,1,"[32736, 104, 21988, 10, 15381, 19386, 7, 2996,..."
1,Mark Zuckerberg to give away 99 percent of his...,1,1,1,"[32736, 11275, 32691, 32749, 9258, 24556, 1722..."
2,GUESS WHAT DAY IT IS TODAY!?,1,1,1,"[32730, 32744, 32728, 32742, 32742, 32691, 327..."
3,How tough are you? Are you this tough? Probabl...,1,1,1,"[32731, 9185, 1962, 30, 249, 178, 32724, 195, ..."
4,He wrote a song about his mom's cancer. That's...,1,1,1,"[32731, 134, 810, 5, 3127, 58, 18, 4649, 32698..."


In [0]:
EMB_DIM = 64
BATCH_SIZE = 64
TRAIN_RATIO = 3

def get_data_set(target_column):
  def gen():
    for index, row in df.iterrows():
      label = 0
      if row[target_column] > 0:
        label = 1
      yield (row["embedding"], label)

  dataset = tf.data.Dataset.from_generator(gen, (tf.int64, tf.int64), (tf.TensorShape([None]), tf.TensorShape([])))

  # Split training and testing data
  train_dataset = dataset.shuffle(len(df))
  test_size = int(len(df)/TRAIN_RATIO)
  train_size = len(df)- test_size
  train_dataset, test_dataset = dataset.take(train_size), dataset.take(-1)
  test_dataset.skip(train_size)

  train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_dataset.output_shapes)
  test_dataset = test_dataset.padded_batch(BATCH_SIZE, test_dataset.output_shapes)
  return (train_dataset, test_dataset)

def train_data(model, train_dataset, test_dataset, epochs=10):
  history = model.fit(train_dataset, epochs=epochs,
                      validation_data=test_dataset, 
                      validation_steps=30)

  test_loss, test_acc = model.evaluate(test_dataset)

  print('Test Loss: {}'.format(test_loss))
  print('Test Accuracy: {}'.format(test_acc))

  return history

Simple CNN

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, EMB_DIM),
		tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

train_dataset, test_dataset = get_data_set(TARGET_COLS[0])
history_lstm = train_data(model, train_dataset, test_dataset)

Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.


Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.


Train on None steps, validate on 30 steps
Epoch 1/10
231/231 [==============================] - 15s 66ms/step - loss: 0.6916 - acc: 0.5527 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/10
231/231 [==============================] - 15s 65ms/step - loss: 0.6895 - acc: 0.5555 - val_loss: 0.6909 - val_acc: 0.5312
Epoch 3/10
231/231 [==============================] - 14s 62ms/step - loss: 0.6881 - acc: 0.5555 - val_loss: 0.6906 - val_acc: 0.5312
Epoch 4/10
231/231 [==============================] - 14s 62ms/step - loss: 0.6871 - acc: 0.5555 - val_loss: 0.6904 - val_acc: 0.5312
Epoch 5/10
231/231 [==============================] - 14s 62ms/step - loss: 0.6863 - acc: 0.5555 - val_loss: 0.6902 - val_acc: 0.5312
Epoch 6/10
231/231 [==============================] - 14s 62ms/step - loss: 0.6857 - acc: 0.5555 - val_loss: 0.6899 - val_acc: 0.5312
Epoch 7/10
231/231 [==============================] - 14s 62ms/step - loss: 0.6852 - acc: 0.5555 - val_loss: 0.6895 - val_acc: 0.5312
Epoch 8/10
2

Simple LSTM

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

train_dataset, test_dataset = get_data_set(TARGET_COLS[0])
history_lstm = train_data(model, train_dataset, test_dataset)

Train on None steps, validate on 30 steps
Epoch 1/10
231/231 [==============================] - 41s 178ms/step - loss: 0.6935 - acc: 0.5139 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/10
231/231 [==============================] - 40s 172ms/step - loss: 0.6899 - acc: 0.5555 - val_loss: 0.6911 - val_acc: 0.5312
Epoch 3/10
231/231 [==============================] - 40s 172ms/step - loss: 0.6886 - acc: 0.5555 - val_loss: 0.6913 - val_acc: 0.5312
Epoch 4/10
231/231 [==============================] - 40s 174ms/step - loss: 0.6882 - acc: 0.5555 - val_loss: 0.6915 - val_acc: 0.5312
Epoch 5/10
231/231 [==============================] - 40s 172ms/step - loss: 0.6878 - acc: 0.5555 - val_loss: 0.6915 - val_acc: 0.5323
Epoch 6/10
231/231 [==============================] - 40s 172ms/step - loss: 0.6878 - acc: 0.5568 - val_loss: 0.6916 - val_acc: 0.5339
Epoch 7/10
231/231 [==============================] - 39s 170ms/step - loss: 0.6875 - acc: 0.5599 - val_loss: 0.6915 - val_acc: 0.5380
Epoch

BiDirectional *LSTM*

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

train_dataset, test_dataset = get_data_set(TARGET_COLS[0])
history_bd = train_data(model, train_dataset, test_dataset)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train on None steps, validate on 30 steps
Epoch 1/10
231/231 [==============================] - 67s 289ms/step - loss: 0.6921 - acc: 0.5323 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/10
231/231 [==============================] - 64s 277ms/step - loss: 0.6891 - acc: 0.5555 - val_loss: 0.6902 - val_acc: 0.5312
Epoch 3/10
231/231 [==============================] - 64s 278ms/step - loss: 0.6869 - acc: 0.5555 - val_loss: 0.6885 - val_acc: 0.5312
Epoch 4/10
231/231 [==============================] - 64s 278ms/step - loss: 0.6824 - acc: 0.5555 - val_loss: 0.6803 - val_acc: 0.5323
Epoch 5/10
231/231 [==============================] - 64s 278ms/step - loss: 0.6588 - acc: 0.6037 - val_loss: 0.6353 - val_acc: 0.6948
Epoch 6/10
231/231 [==============================] - 65s 279ms/step - loss: 0.5865 - acc: 0.7129 - val_loss: 0.5579 - val_acc: 0.7302
Epoch 7/10
231/231 [==============================] - 64s 278ms/step - loss: 0.5035 - acc: 0.7578 - val_loss: 0.4871 - val_acc: 0.7719
Epoch


BiDirectional LSTM with different labels

"open2open", "overnight"



In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

train_dataset, test_dataset = get_data_set(TARGET_COLS[1])
train_data(model, train_dataset, test_dataset)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

train_dataset, test_dataset = get_data_set(TARGET_COLS[2])
train_data(model, train_dataset, test_dataset)

Train on None steps, validate on 30 steps
Epoch 1/10
231/231 [==============================] - 66s 287ms/step - loss: 0.6927 - acc: 0.5204 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/10
231/231 [==============================] - 64s 278ms/step - loss: 0.6890 - acc: 0.5555 - val_loss: 0.6906 - val_acc: 0.5312
Epoch 3/10
231/231 [==============================] - 64s 279ms/step - loss: 0.6872 - acc: 0.5555 - val_loss: 0.6896 - val_acc: 0.5312
Epoch 4/10
231/231 [==============================] - 64s 279ms/step - loss: 0.6845 - acc: 0.5555 - val_loss: 0.6846 - val_acc: 0.5312
Epoch 5/10
231/231 [==============================] - 65s 283ms/step - loss: 0.6708 - acc: 0.5756 - val_loss: 0.6551 - val_acc: 0.6729
Epoch 6/10
231/231 [==============================] - 67s 292ms/step - loss: 0.6046 - acc: 0.6935 - val_loss: 0.5612 - val_acc: 0.7318
Epoch 7/10
231/231 [==============================] - 68s 296ms/step - loss: 0.5086 - acc: 0.7535 - val_loss: 0.4724 - val_acc: 0.7865
Epoch

More epochs, 20

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

train_dataset, test_dataset = get_data_set(TARGET_COLS[0])
train_data(model, train_dataset, test_dataset, epochs=20)

Train on None steps, validate on 30 steps
Epoch 1/20
231/231 [==============================] - 68s 296ms/step - loss: 0.6919 - acc: 0.5342 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/20
231/231 [==============================] - 71s 309ms/step - loss: 0.6887 - acc: 0.5555 - val_loss: 0.6903 - val_acc: 0.5312
Epoch 3/20
231/231 [==============================] - 73s 317ms/step - loss: 0.6867 - acc: 0.5555 - val_loss: 0.6882 - val_acc: 0.5312
Epoch 4/20
231/231 [==============================] - 73s 317ms/step - loss: 0.6809 - acc: 0.5564 - val_loss: 0.6751 - val_acc: 0.5625
Epoch 5/20
231/231 [==============================] - 73s 317ms/step - loss: 0.6433 - acc: 0.6430 - val_loss: 0.6074 - val_acc: 0.7031
Epoch 6/20
231/231 [==============================] - 74s 319ms/step - loss: 0.5458 - acc: 0.7314 - val_loss: 0.5046 - val_acc: 0.7651
Epoch 7/20
231/231 [==============================] - 68s 295ms/step - loss: 0.4547 - acc: 0.7919 - val_loss: 0.4179 - val_acc: 0.8172
Epoch

Deeper architecture, extra hidden layer

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

train_dataset, test_dataset = get_data_set(TARGET_COLS[0])
train_data(model, train_dataset, test_dataset, epochs=20)

Train on None steps, validate on 30 steps
Epoch 1/20
231/231 [==============================] - 67s 288ms/step - loss: 0.6921 - acc: 0.5428 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/20
231/231 [==============================] - 63s 275ms/step - loss: 0.6893 - acc: 0.5555 - val_loss: 0.6906 - val_acc: 0.5312
Epoch 3/20
231/231 [==============================] - 64s 277ms/step - loss: 0.6873 - acc: 0.5555 - val_loss: 0.6893 - val_acc: 0.5312
Epoch 4/20
231/231 [==============================] - 66s 286ms/step - loss: 0.6830 - acc: 0.5555 - val_loss: 0.6816 - val_acc: 0.5312
Epoch 5/20
231/231 [==============================] - 68s 294ms/step - loss: 0.6583 - acc: 0.5818 - val_loss: 0.6408 - val_acc: 0.6823
Epoch 6/20
231/231 [==============================] - 67s 291ms/step - loss: 0.5780 - acc: 0.7147 - val_loss: 0.5424 - val_acc: 0.7359
Epoch 7/20
231/231 [==============================] - 69s 299ms/step - loss: 0.4818 - acc: 0.7752 - val_loss: 0.4550 - val_acc: 0.7937
Epoch